In [1]:
import os
import re
import sys
sys.path.append('../')
import json
import pywt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import simpson
from sklearn.linear_model import RidgeClassifier
from tqdm import tqdm

from transform import TimeSeriesTransform

In [2]:
cfg = json.load(open("../config/config.json"))

ts_trans = TimeSeriesTransform(cfg)

In [3]:
INSTANCES_DIR = '../data/linear_acuator/instances/'
INFERENCE_DIR = '../data/linear_acuator/inference/'
STATES = ['normal', 
            'backlash1', 'backlash2',
            'lackLubrication1', 'lackLubrication2',
            'spalling1', 'spalling2', 'spalling3', 'spalling4', 'spalling5', 'spalling6', 'spalling7', 'spalling8']
LOADS= ['20kg', '40kg', '-40kg']

In [28]:
load = '20kg'
filenames_20kg = [os.listdir(os.path.join(INFERENCE_DIR, load, state)) for state in STATES]
filenames_20kg = [filename for sublist in filenames_20kg for filename in sublist]

load = '40kg'
filenames_40kg = [os.listdir(os.path.join(INFERENCE_DIR, load, state)) for state in STATES]
filenames_40kg = [filename for sublist in filenames_40kg for filename in sublist]

load = '-40kg'
filenames_m40kg = [os.listdir(os.path.join(INFERENCE_DIR, load, state)) for state in STATES]
filenames_m40kg = [filename for sublist in filenames_m40kg for filename in sublist]

In [36]:
def get_X_y(filenames, load):
    X, y = [], []
    for filename in filenames:
        load_num = load[:-2]
        state = re.match(fr'(.*)_{load_num}', filename).group(1)
        df = pd.read_csv(os.path.join(INFERENCE_DIR, load, state, filename))
        tmp_cur = ts_trans.smoothing(ts_df=df, field='current')
        # tmp_pos = ts_transform.smoothing(ts_df=df, field='position_error')
        X.append(tmp_cur)
        y.append(state)
    return X, y

In [38]:
X_20kg, y_20kg = get_X_y(filenames_20kg, load='20kg')
X_40kg, y_40kg = get_X_y(filenames_40kg, load='40kg')
X_m40kg, y_m40kg = get_X_y(filenames_m40kg, load='-40kg')

In [39]:
lower_bound = 1   # Lower bound of the range (inclusive)
upper_bound = 1000  # Upper bound of the range (exclusive)
n_kernels = 1000
np.random.seed(42)  # for reproducibility
random_scales = np.random.uniform(lower_bound, upper_bound, n_kernels)
wavelet = 'morl'  # Morlet wavelet for CWT

In [40]:
def raven(X):
    raven_results = []
    for i in tqdm(range(len(X))):
        coefficients, _ = pywt.cwt(X[i], random_scales, wavelet, sampling_period=1/400)
        integral_simps = simpson(coefficients)
        raven_results.append(integral_simps)
    return raven_results

In [41]:
raven_results_20kg = raven(X_20kg)
raven_results_40kg = raven(X_40kg)
raven_results_m40kg = raven(X_m40kg)

100%|██████████| 195/195 [01:48<00:00,  1.80it/s]


In [45]:
ridge = RidgeClassifier()
ridge.fit(raven_results_20kg, y_20kg)
print(f'20kg: {ridge.score(raven_results_20kg, y_20kg)}')

20kg: 0.9692307692307692


In [46]:
ridge = RidgeClassifier()
ridge.fit(raven_results_40kg, y_40kg)
print(f'40kg: {ridge.score(raven_results_40kg, y_40kg)}')

40kg: 0.9637305699481865


In [47]:
ridge = RidgeClassifier()
ridge.fit(raven_results_m40kg, y_m40kg)
print(f'-40kg: {ridge.score(raven_results_m40kg, y_m40kg)}')

-40kg: 0.9743589743589743


In [48]:
ridge.predict(raven_results_m40kg)

array(['normal', 'normal', 'normal', 'normal', 'spalling3', 'normal',
       'normal', 'normal', 'normal', 'normal', 'normal', 'normal',
       'normal', 'normal', 'normal', 'backlash1', 'backlash1',
       'backlash1', 'backlash1', 'backlash1', 'backlash1', 'backlash1',
       'backlash1', 'backlash1', 'backlash1', 'backlash1', 'backlash1',
       'backlash1', 'backlash1', 'backlash1', 'backlash2', 'backlash2',
       'backlash2', 'backlash2', 'backlash2', 'backlash2', 'backlash2',
       'backlash2', 'backlash2', 'backlash2', 'backlash2', 'backlash2',
       'backlash2', 'backlash2', 'backlash2', 'lackLubrication1',
       'lackLubrication1', 'lackLubrication1', 'lackLubrication1',
       'lackLubrication1', 'lackLubrication1', 'lackLubrication1',
       'lackLubrication1', 'lackLubrication1', 'lackLubrication1',
       'lackLubrication1', 'lackLubrication1', 'lackLubrication1',
       'lackLubrication1', 'lackLubrication1', 'lackLubrication2',
       'lackLubrication2', 'lackLubrica

In [50]:
np.array(y_m40kg)

array(['normal', 'normal', 'normal', 'normal', 'normal', 'normal',
       'normal', 'normal', 'normal', 'normal', 'normal', 'normal',
       'normal', 'normal', 'normal', 'backlash1', 'backlash1',
       'backlash1', 'backlash1', 'backlash1', 'backlash1', 'backlash1',
       'backlash1', 'backlash1', 'backlash1', 'backlash1', 'backlash1',
       'backlash1', 'backlash1', 'backlash1', 'backlash2', 'backlash2',
       'backlash2', 'backlash2', 'backlash2', 'backlash2', 'backlash2',
       'backlash2', 'backlash2', 'backlash2', 'backlash2', 'backlash2',
       'backlash2', 'backlash2', 'backlash2', 'lackLubrication1',
       'lackLubrication1', 'lackLubrication1', 'lackLubrication1',
       'lackLubrication1', 'lackLubrication1', 'lackLubrication1',
       'lackLubrication1', 'lackLubrication1', 'lackLubrication1',
       'lackLubrication1', 'lackLubrication1', 'lackLubrication1',
       'lackLubrication1', 'lackLubrication1', 'lackLubrication2',
       'lackLubrication2', 'lackLubricatio